# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Подготовка данных

### Обзор данных

Импортируем необходимые библиотеки

In [1]:
import pandas as pd
import numpy as np
from scipy import stats as st 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

Прочитайте файлы формата .csv из папки /datasets и сохраним их в переменных df_0, df_1, df_2

In [2]:
data_0 = pd.read_csv('/datasets/geo_data_0.csv')
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
data_2 = pd.read_csv('/datasets/geo_data_2.csv')

Выведем на экран первые 5 строк таблиц, а также посмотрим общую информацию

In [3]:
data_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [4]:
data_1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [5]:
data_2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [6]:
data_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [7]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [8]:
data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


В таблицах присутствуют пять столбцов. Четыре стобца имеют тип данных `float`, один тип данных `object`.

Согласно документации к данным:

- `id` — уникальный идентификатор скважины;
- `f0`, `f1`, `f2`— три признака точек (неважно, что они означают, но сами признаки значимы);
- `product` — объём запасов в скважине (тыс. баррелей).

### Предобработка данных

Столбец `id` содержит информацию об уникальном идентификаторе скважины. Он не несет особой ценности для изучения модели, но может быть полезен при поиске дубликатов.

In [9]:
print(data_0['id'].value_counts().sum())
print(data_1['id'].value_counts().sum())
print(data_2['id'].value_counts().sum())

100000
100000
100000


В столбце `id` получено 10000 уникальных значений, что равняется общему количеству строк, значит дубликатов нет. После проверки столбец можно удалить.

In [10]:
df_0 = data_0.drop('id', axis=1)
df_0.head()

,f0,f1,f2,product
0,0.705745,-0.497823,1.221170,105.280062
1,1.334711,-0.340164,4.365080,73.037750
2,1.022732,0.151990,1.419926,85.265647
3,-0.032172,0.139033,2.978566,168.620776
4,1.988431,0.155413,4.751769,154.036647


In [11]:
df_1 = data_1.drop('id', axis=1)
df_2 = data_2.drop('id', axis=1)

###  Разбитие данных на выборки

In [12]:
df_0.shape # изучим размер исходной таблицы

(100000, 4)

In [13]:
# обучающая=0.75, валидационная = 0.25
df_0_train, df_0_valid = train_test_split(df_0, test_size=0.25, random_state=12345) 

In [14]:
print(df_0_train.shape)
print(df_0_valid.shape)

(75000, 4)
(25000, 4)


In [15]:
# аналогично разобьем выборки в оставшихся таблицах
df_1_train, df_1_valid = train_test_split(df_1, test_size=0.25, random_state=12345) 
df_2_train, df_2_valid = train_test_split(df_2, test_size=0.25, random_state=12345) 

### Масштабирование признаков

Один из методов масштабирования — стандартизация данных. Преобразуем обучающую, валидационную и тестовую выборки функцией transform(). Определим признаки и целевой признак для выборок и стандартизируем выборки.

In [16]:
features_train_0, features_valid_0 = df_0_train.drop('product', axis=1), df_0_valid.drop('product', axis=1)
target_train_0, target_valid_0 = df_0_train['product'], df_0_valid['product']

features_train_1, features_valid_1 = df_1_train.drop('product', axis=1), df_1_valid.drop('product', axis=1)
target_train_1, target_valid_1 = df_1_train['product'], df_1_valid['product']

features_train_2, features_valid_2 = df_2_train.drop('product', axis=1), df_2_valid.drop('product', axis=1)
target_train_2, target_valid_2 = df_2_train['product'], df_2_valid['product']

In [17]:
numeric = ['f0', 'f1', 'f2']

In [18]:
scaler = StandardScaler()
scaler.fit(features_train_0[numeric])

StandardScaler()

In [19]:
features_train_0[numeric] = scaler.transform(features_train_0[numeric])
features_valid_0[numeric] = scaler.transform(features_valid_0[numeric])

In [20]:
features_train_0.head()

,f0,f1,f2
27212,-0.544828,1.390264,-0.094959
7866,1.455912,-0.480422,1.209567
62041,0.260460,0.825069,-0.204865
70185,-1.837105,0.010321,-0.147634
82230,-1.299243,0.987558,1.273181


## Обучение и проверка модели

Напишем функцию обучения модели линейной регрессии и предсказания размера запаса в скважине

In [21]:
def lin_reg(features_train, target_train, features_valid):
    model = LinearRegression()
    model.fit(features_train, target_train)
    predicted = model.predict(features_valid)   
    return predicted

In [22]:
predicted_0 = lin_reg(features_train_0, target_train_0, features_valid_0)
predicted_1 = lin_reg(features_train_1, target_train_1, features_valid_1)
predicted_2 = lin_reg(features_train_2, target_train_2, features_valid_2)

Найдем средний запас сырья, средний запас предсказанного сырья и RMSE модели

In [23]:
def find_answers(target_valid, predicted):
    print('Средний запас сырья =', round(target_valid.mean(), 2))
    print('Предсказанный средний запас сырья =', round(predicted.mean(), 2))
    print("RMSE =", round(mean_squared_error(target_valid, predicted) ** 0.5, 2)) 

In [24]:
find_answers(target_valid_0, predicted_0)

Средний запас сырья = 92.08
Предсказанный средний запас сырья = 92.59
RMSE = 37.58


In [25]:
find_answers(target_valid_1, predicted_1)

Средний запас сырья = 68.72
Предсказанный средний запас сырья = 68.73
RMSE = 0.89


In [26]:
find_answers(target_valid_2, predicted_2)

Средний запас сырья = 94.88
Предсказанный средний запас сырья = 94.97
RMSE = 40.03


Для предсказания мы использовали линейную модель регрессии. 
Самый большой средний объем запаса сырья в третьем регионе, в первом регионе запасов чуть меньше. При этом второй регион значительно выигрывает по показателю **RMSE**

## Подготовка к расчёту прибыли

При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
Бюджет на разработку скважин в регионе — 10 млрд рублей.
При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.

In [27]:
budget = 10000000000
income = 450000
dots = 200

In [28]:
optimal_product = budget / income / dots

In [29]:
print('Объём сырья для безубыточной разработки новой скважины', round(optimal_product,2))

Объём сырья для безубыточной разработки новой скважины 111.11


Рассчитаем прибыль с лучших 200 скважин в каждом регионе

In [30]:
def profit(target_valid, predicted):
    predicted_sort = pd.Series(predicted).sort_values(ascending=False)[:dots]
    target_valid_sort = (target_valid.reset_index(drop = True)[predicted_sort.index])
    sum_dots = target_valid_sort.sum() 
    profit = round((sum_dots * income) - budget,2)
    return profit

In [31]:
profit_0 = profit(target_valid_0, predicted_0)
profit_1 = profit(target_valid_1, predicted_1)
profit_2 = profit(target_valid_2, predicted_2)
print(profit_0)
print(profit_1)
print(profit_2)

3320826043.14
2415086696.68
2710349963.6


Минимальное количество баррелей нефти для безубыточной разработки составило 111.11

В тоже время максимальное средний объем в регионах равен 94.88. Самый низкий показатель среднего запаса сырья во втором регионе, но, при этом наша модель очень хорошо прогнозирует запасы скважин именно в этом регионе.

## Расчёт прибыли и рисков 

После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.
Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

Напишем функцию, в которой сгенерируем 1000 выборок, чтобы найти распределение прибыли

In [32]:
def confidence_interval(target, predicted):
    state = np.random.RandomState(12345)

    samples = []
    for i in range(1000):
        sample = pd.Series(predicted).sample(replace=True, random_state=state, n = 500)
        samples.append(profit(target, sample))
    samples = pd.Series(samples)
    
    return samples 

Напишем функции вычисляющие среднюю выручку, 95%-й доверительный интервал и риск убытков.

In [49]:
def mean_revenue(value):
    print("Средняя выручка:", round(value.mean(), 2))
    
def interval_95(value):
    print("95%-й Доверительный интервал от", round(value.quantile(0.025), 2), "до", round(value.quantile(0.975), 2))
    
def risk_of_loss(value):
    print("Риск убытков:", round(st.percentileofscore(value, 0),2), '%')

In [50]:
interval_0 = confidence_interval(target_valid_0, predicted_0)
interval_1 = confidence_interval(target_valid_1, predicted_1)
interval_2 = confidence_interval(target_valid_2, predicted_2)

In [51]:
mean_revenue(interval_0)
interval_95(interval_0)
risk_of_loss(interval_0)

Средняя выручка: 396164984.8
95%-й Доверительный интервал от -111215545.89 до 909766941.55
Риск убытков: 6.9 %


In [52]:
mean_revenue(interval_1)
interval_95(interval_1)
risk_of_loss(interval_1)

Средняя выручка: 456045105.79
95%-й Доверительный интервал от 33820509.4 до 852289453.86
Риск убытков: 1.5 %


In [53]:
mean_revenue(interval_2)
interval_95(interval_2)
risk_of_loss(interval_2)

Средняя выручка: 404403866.57
95%-й Доверительный интервал от -163350413.39 до 950359574.93
Риск убытков: 7.6 %


По предварительным оценкам второй регион не подходил под выбор для разработки скважин, так как средние запасы региона значительно ниже запасов в других регионах, а также он не подходит под условие безубыточности. Но при этом, применив технику Bootstrap с 1000 выборок, мы обнаружили, что данный регион единственных, подходящий под условия разработки (вероятность убытков меньше 2.5%). 

## Вывод

- На первом этапе мы изучили содержимое таблиц, подготовили данные к работе, разбили данные на тестовую и валидационную выборки и масштабировали признаки. 

- Затем обучили модель методом линейной регрессии, нашли среднее количество добываемого сырья в каждом регионе и проверили качество модели. Предсказанный средний запас сырья в первом регионе = 92.59, во втором = 68.73, в третьем = 94.97, RMSE в первом регионе = 37.58, во втором = 0.89, в третьем = 40.03.

- Далее мы нашли объём сырья для безубыточной разработки новой скважины равен 111.11, затем написали функцию подсчета прибыли для выбранных скважи

- На следующем этапе написали функцию, в которой сгенерировали 1000 выборок, чтобы найти распределение прибыли, а также функции, вычисляющие среднюю выручку, 95%-й доверительный интервал и риск убытков. Мы обнаружили, что второй регион единственных, подходящий под условия разработки (вероятность убытков меньше 2.5%). Средняя выручка в нем = 456045105.79, доверительный интервал = от 33820509.4 до 852289453.86